In [1]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

/home/lucapug/my-llm-zoomcamp/hw4/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

In [3]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [5]:
df.shape

(1830, 5)

In [6]:
df = df.iloc[:300]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   answer_llm   300 non-null    object
 1   answer_orig  300 non-null    object
 2   document     300 non-null    object
 3   question     300 non-null    object
 4   course       300 non-null    object
dtypes: object(5)
memory usage: 11.8+ KB


In [8]:
model_name = 'multi-qa-mpnet-base-dot-v1'

In [9]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [10]:
answer_llm = df.iloc[0].answer_llm
v_answer_llm = embedding_model.encode(answer_llm)

v_answer_llm[:5]

array([-0.42244673, -0.22485583, -0.32405847, -0.28475878,  0.00725701],
      dtype=float32)

### ans.1 -0.42

In [11]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)
    
    return v_llm.dot(v_orig)

In [12]:
results_gpt4o_mini = df.to_dict(orient='records')

In [13]:
evaluations = []

for record in tqdm(results_gpt4o_mini):
    score = compute_similarity(record)
    evaluations.append(score)

100%|████████████████████████████████████████████████████████| 300/300 [00:10<00:00, 28.68it/s]


In [14]:
evaluations[0]

17.515999

In [15]:
max(evaluations)

39.476017

In [16]:
df['dot_prod'] = evaluations

In [17]:
df.dot_prod.describe()

count    300.000000
mean      27.495996
std        6.384743
min        4.547922
25%       24.307846
50%       28.336861
75%       31.674309
max       39.476017
Name: dot_prod, dtype: float64

### ans.2 31.67

In [18]:
def compute_cosine(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    v_norm_llm = v_llm/np.sqrt((v_llm * v_llm).sum())
    v_norm_orig = v_orig/np.sqrt((v_orig * v_orig).sum())
    
    return v_norm_llm.dot(v_norm_orig)

In [19]:
evaluations2 = []

for record in tqdm(results_gpt4o_mini):
    score = compute_cosine(record)
    evaluations2.append(score)

100%|████████████████████████████████████████████████████████| 300/300 [00:10<00:00, 29.66it/s]


In [20]:
max(evaluations2)

0.9587959

In [21]:
df['cosine'] = evaluations2

In [22]:
df.cosine.describe()

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine, dtype: float64

### ans.3 0.83

In [32]:
id = df.iloc[10].document
id

'5170565b'

In [33]:
# answer_llm = df.loc[df['document']=='5170565b'].answer_llm  #5 answers refer to the same ID
answer_llm = df.iloc[10].answer_llm
answer_llm

"Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded."

In [34]:
answer_orig = df.iloc[10].answer_orig
answer_orig

'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.'

In [45]:
from rouge import Rouge
rouge_scorer = Rouge()


scores = []
for i in range(len(df)):
    answer_llm = df.iloc[i].answer_llm
    answer_orig = df.iloc[i].answer_orig
    scores.append(rouge_scorer.get_scores(answer_llm, answer_orig)[0])

In [47]:
scores[10]

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

#### ans.4 0.45

In [53]:
rouge_1 = scores[10]['rouge-1']['f']
rouge_2 = scores[10]['rouge-2']['f']
rouge_l = scores[10]['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [54]:
rouge_avg

0.35490034990035496

#### ans.5 0.35

In [55]:
len(scores)

300

In [58]:
scores[0:2]

[{'rouge-1': {'r': 0.061224489795918366,
   'p': 0.21428571428571427,
   'f': 0.09523809178130524},
  'rouge-2': {'r': 0.017543859649122806,
   'p': 0.07142857142857142,
   'f': 0.028169010918468917},
  'rouge-l': {'r': 0.061224489795918366,
   'p': 0.21428571428571427,
   'f': 0.09523809178130524}},
 {'rouge-1': {'r': 0.08163265306122448,
   'p': 0.26666666666666666,
   'f': 0.12499999641113292},
  'rouge-2': {'r': 0.03508771929824561,
   'p': 0.13333333333333333,
   'f': 0.05555555225694465},
  'rouge-l': {'r': 0.061224489795918366, 'p': 0.2, 'f': 0.09374999641113295}}]

In [61]:
temp=[scores[i]['rouge-1']['f'] for i in range(len(df))]

In [64]:
temp[0]

0.09523809178130524

In [67]:
rouge_1 = [scores[i]['rouge-1']['f'] for i in range(len(df))]
rouge_2 = [scores[i]['rouge-2']['f'] for i in range(len(df))]
rouge_l = [scores[i]['rouge-l']['f'] for i in range(len(df))]
#rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [68]:
rouge_1[0:3]

[0.09523809178130524, 0.12499999641113292, 0.41558441095631643]

In [78]:
import numpy as np

data = np.array([rouge_1, rouge_2, rouge_l])
rouge_avg=np.average(data, axis=0)

In [79]:
rouge_avg[0]

0.07288173149369313

In [80]:
(scores[0]['rouge-1']['f']+scores[0]['rouge-2']['f']+scores[0]['rouge-l']['f'])/3

0.07288173149369313

In [83]:
df1 = pd.DataFrame({'rouge_1':rouge_1, 'rouge_2':rouge_2, 'rouge_l':rouge_l, 'rouge_avg':rouge_avg})

In [84]:
df1.head()

,rouge_1,rouge_2,rouge_l,rouge_avg
0,0.095238,0.028169,0.095238,0.072882
1,0.125000,0.055556,0.093750,0.091435
2,0.415584,0.177778,0.389610,0.327658
3,0.216216,0.047059,0.189189,0.150821
4,0.142076,0.033898,0.120219,0.098731


In [86]:
df1.rouge_2.mean()

0.20696501983423318

#### ans.6 0.20